In [1]:
# conda env export > environment.yml

In [ ]:
# import torch
# import gc

# # ✅ 메모리 초기화 (강제 GC + CUDA 캐시 초기화)
# gc.collect()
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.ipc_collect()

In [2]:
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from transformers import AutoModelForCausalLM, AutoTokenizer
from pydantic import PrivateAttr
import torch

if torch.cuda.is_available():
    print("CUDA 사용 가능 (GPU 사용 가능)")
    print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 사용 중인 디바이스 이름: {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA 사용 불가 (CPU만 사용 가능)")


CUDA 사용 가능 (GPU 사용 가능)
사용 가능한 GPU 개수: 1
현재 사용 중인 디바이스 이름: NVIDIA GeForce RTX 4060


In [ ]:
class ChatEXAONE(BaseChatModel):
    # 여기에 미리 선언해줘야 함!!
    _tokenizer: any = PrivateAttr()
    _model: any = PrivateAttr()

    def __init__(self, model_name="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"):
        super().__init__()
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            device_map="auto"
        )

    @property
    def _llm_type(self) -> str:
        return "chat-exaone"

    def _generate(self, messages, stop=None):
        chat_template = []
        for msg in messages:
            if isinstance(msg, SystemMessage):
                chat_template.append({"role": "system", "content": msg.content})
            elif isinstance(msg, HumanMessage):
                chat_template.append({"role": "user", "content": msg.content})
            elif isinstance(msg, AIMessage):
                chat_template.append({"role": "assistant", "content": msg.content})

        input_ids = self._tokenizer.apply_chat_template(
            chat_template,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        outputs = self._model.generate(
            input_ids.to("cuda"),
            eos_token_id=self._tokenizer.eos_token_id,
            max_new_tokens=256,
            do_sample=False,
        )
        decoded = self._tokenizer.decode(outputs[0], skip_special_tokens=True)
        return AIMessage(content=decoded)


In [5]:

# ChatEXAONE 클래스는 이미 네가 주었던 코드라고 가정하고, 바로 사용!

# 1. 모델 인스턴스 생성
chat_model = ChatEXAONE()

# 2. 사용자 질문 만들기
user_message = HumanMessage(content="너를 소개해봐")

# 3. 모델에 질문 보내고 답변 받기
response = chat_model._generate(messages=[user_message])

# 4. 답변 출력
print("🤖 모델의 답변:")
print(response.content)


Loading checkpoint shards: 100%|██████████| 7/7 [00:14<00:00,  2.11s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


KeyboardInterrupt: 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Choose your prompt
prompt = "Explain how wonderful you are"  # English example
prompt = "스스로를 자랑해 봐"       # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))


c:\Users\geon1\anaconda3\envs\EXAONE_langchain_wrapper_cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\geon1\anaconda3\envs\EXAONE_langchain_wrapper_cuda\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\geon1\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode 

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]스스로를 자랑해 봐
[|assistant|]저는 LG AI Research에서 개발된 EXAONE 모델로서, 뛰어난 자연어 처리 능력을 바탕으로 다양한 언어 작업을 수행할 수 있습니다. 지속적인 학습을 통해 사용자의 질문에 정확하고 신속하게 응답하며, 복잡한 정보도 명확하게 전달하는 데 초점을 맞춥니다. 이러한 기술 덕분에 다양한 분야에서 효율적인 지원을 제공하며, 사용자 경험 향상에 기여하고 있습니다.[|endofturn|]
